# Task 1

In [ ]:
'https://foobar.withgoogle.com/'

In [42]:
def solution(i):
    primes = getPrimeString()
    id = primes[i:i+5]
    return id

In [43]:
def getPrimeString():
    primes = ""
    prime = 2

    while len(primes) < 10005:
        primes += str(prime)
        prime +=1
        while not isPrime(prime):
            prime += 1
    return primes    

In [44]:
def isPrime(n):
    if n == 1:
        return False
    i = 2
    while i*i <= n:
        if n % i ==0:
            return False
        i +=1
    return True

# Task 2

In [81]:
class numbers:
    def __init__(self, elevator):
        self.full = elevator
        split = list(map(int, elevator.strip().split('.')))
        self.major = split[0]
        self.minor = split[1] if len(split) > 1 else -1
        self.revision = split[2] if len(split) > 2 else -1
    def __lt__(self, other):
        if self.major < other.major: return True
        if self.major > other.major: return False
        if self.minor < other.minor: return True
        if self.minor > other.minor: return False
        if self.revision < other.revision: return True
        if self.revision > other.revision: return False

In [85]:
def solution(l):
    els = []
    for elevator in l:
        els.append(numbers(elevator))
    els.sort()
    answer = [el.full for el in els]
    return answer

In [425]:
from fractions import Fraction

def solution(pegs):
    #calculate the differences between pegs
    diff = [pegs[i+1] - pegs[i] for i in range(len(pegs)-1)]
    #get number of pegs and if this is even (solution differs based on this)
    size = len(pegs)
    even = True if size % 2 == 0 else False
    #calculate y based on the differences
    if even:
        y = sum(diff[::2]) - sum(diff[1::2])
        y = float(y) / 3
    else:
        y = sum(diff[::2]) - sum(diff[1::2])
        y = y / 1
        
    #meet the constraint that x = 2y
    x = y * 2
    #work through all gears to make sure they are positive and > 1too
    current = x
    for i in range(len(diff)):
        distance = diff[i]
        next = distance - current
        if ((current <1) or (next <1)):
            return [-1,-1]
        else:
            current = next
    #only return value if the sizes are positive
    if (x >= 2):
        ans = Fraction(x).limit_denominator() 
        return ans.numerator, ans.denominator
    else:
        return -1,-1


In [431]:
pegs = [4,30, 50,80]
solution(pegs)

(24, 1)

In [428]:
pegs = [4,17,50]
solution(pegs)

[-1, -1]

In [429]:
pegs=[1,8]
solution(pegs)

(14, 3)

## task 2.2

In [598]:
def postorder_traverse(h, target):
    ## the number of integers in the tree
    size = 2**h-1
    
    #define the parent and child nodes (assume only 1 number in tree)
    parent_node = size
    right_current = size
    left_current = size
    
    # the # of subtrees  
    layers = h - 1
    
    # if the parent node is the target node then there is no parent
    if (target > 0 and target <= size):
        if parent_node == target:
            return -1
    
    # loop through posttraversal   
    while(True):
        # assign the right and left child nodes (the left node follows specific calculation)
        right_current = parent_node - 1
        level = ((2**layers)-1)
        left_current = right_current - level
        
        # if the child node is the target then get the parent
        if (target == left_current or target == right_current):
            return parent_node
        
        # traverse down the tree
        if (target < left_current):
            parent_node = left_current
            layers = layers - 1
        if (target > left_current):
            parent_node = right_current
            layers = layers - 1
    return -1

def solution(h, q):
    answers = []
    for num in q:
        answers.append(postorder_traverse(h,num))
    return answers

In [600]:
solution(4, [1,7,4])

[3, 15, 6]

## task 3

In [881]:
import itertools
import copy

def solution(map):
    n = len(map)
    m = len(map[0])
    shape = (n,m)
    #row and column identifiers
    rows = [str(x) for x in range(n)]
    cols = [str(x) for x in range(m)]
    #create nodes
    nodes = [c+'-'+r for (c,r) in itertools.product(rows,cols)]
    # add weights 
    impossible_weight = n*m
    weights = [(node*impossible_weight)+1 for row in map for node in row]

    results = []
    for i, weight in enumerate(weights):
        if weight > impossible_weight:
            #create new weights removing one wall
            new_weights = copy.deepcopy(weights)
            new_weights[i] = new_weights[i] - impossible_weight
            #create graph and run djikstras
            graph = Graph(nodes, new_weights, shape)
            shortest_path = dijkstra_algorithm(graph, nodes[0])
            path = shortest_path.get(nodes[-1])+1
            results.append(path)
    answer = min(results)
    return(answer)

class Graph():
    def __init__(self, nodes, weights,shape):
        self.nodes = nodes
        self.graph = self.construct_graph(nodes, weights, shape)
        
    def construct_graph(self, nodes, weights, shape):
        n,m = shape
        # create the graph from the new weights
        in_graph = {}
        #create adj list fr each node
        for node in nodes:
            in_graph[node] = {}
        for i, node in enumerate(nodes):
            #all but last node
            if i < len(nodes) - 1:
                # all nodest no in last column connect to the right
                if (i+1) % m != 0:
                    in_graph[node][nodes[i+1]] = max(weights[i], weights[i+1])
                # all nodes not in last row connect to below
                if i < len(nodes) - m :
                    in_graph[node][nodes[i+m]] = max(weights[i], weights[i+m])

        for node, edges in in_graph.items():
            for adjacent_node, value in edges.items():
                if in_graph[adjacent_node].get(node, False) == False:
                    in_graph[adjacent_node][node] = value
        return in_graph
    
    def get_nodes(self):
        return self.nodes
    
    def get_outgoing_edges(self, node):
        connections = []
        for out_node in self.nodes:
            if self.graph[node].get(out_node, False) != False:
                connections.append(out_node)   
        return connections
    
    def value(self, node1, node2):
        return self.graph[node1][node2]
        

def dijkstra_algorithm(graph, start_node):
    unvisited_nodes = list(graph.get_nodes())
    shortest_path = {}
    previous_nodes = {}
    max_value = float("inf")
    
    for node in unvisited_nodes:
        shortest_path[node] = max_value
    shortest_path[start_node] = 0
        
    while unvisited_nodes:
        current_min_node = None
        for node in unvisited_nodes:
            if current_min_node == None:
                current_min_node = node
                        
            elif shortest_path[current_min_node] > shortest_path[node]:
                current_min_node = node
                
        neighbors = graph.get_outgoing_edges(current_min_node)
        for neighbor in neighbors:
            tentative_value = shortest_path[current_min_node] + graph.value(current_min_node, neighbor)
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                previous_nodes[neighbor] = current_min_node
        unvisited_nodes.remove(current_min_node)
    return shortest_path



In [882]:
map = [[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0]]
solution(map)

11

In [860]:
map = [[0, 1, 1, 0], [0, 0, 0, 1], [1, 1, 0, 0], [1, 1, 1, 0]]
solution(map)

7

## task 3.2

In [964]:
n = 200

In [969]:
def solution(n):
    #create DP table
    steps = [[0 for _ in range(n+1)] for _ in range(n+1)]
    steps[0][0] = 1
    #fill this in sequentially moving across columns then rows
    for brick in range(1, n+1):
        for height in range(n+1):
            steps[brick][height] = steps[brick-1][height]
            if height >= brick:
                steps[brick][height] += steps[brick-1][height-brick]
    return steps[n][n] -1

In [970]:
solution(200)

487067745

In [1013]:
15//2

7

## task 3.3

In [1108]:
def solution(n):
    n = int(n)
    steps = 0
    while n > 1:
        if n % 2 == 0:
            n = n/2
        elif ((n!=3) or (bin(n)[-3:] == '111')):
            n += 1
        else:
            n -=1
        steps +=1 
    return steps

In [1117]:
n = 15


In [1118]:
solution(n)


5

## task 4.1

In [14]:
from fractions import math
import itertools 

def loops(x,y):
    total = x + y
    great_common_denom = math.gcd(x,y)
    z = total / great_common_denom
    z = int(z)
    return bool((z & (z-1)))

def createGraph(banana_list):
    ## identify infinite loops
    nodes = {}
    ## give each node a unique name in case there are repeat numbers
    node_names = [str(i) for i in range(len(banana_list))]
    for i, node in enumerate(banana_list):
        nodes[node_names[i]] = node

    combs = list(itertools.combinations(node_names,2))
    inf = []
    for comb in combs:
        if loops(nodes[comb[0]],nodes[comb[1]]) == True:
            inf.append(comb)

    ##create matrix    
    graph = [[0 for i in range(len(banana_list))] for i in range(len(banana_list))]

    for i in inf:
        graph[node_names.index(i[0])][node_names.index(i[1])] = 1
        graph[node_names.index(i[1])][node_names.index(i[0])] = 1

    return graph

# Python program to findmaximal Bipartite matching.

class GFG:
	def __init__(self,graph):
		
		# residual graph
		self.graph = graph
		self.node_a = len(graph)
		self.node_b = len(graph[0])

	# A DFS based recursive function
	# that returns true if a matching
	# for vertex u is possible
	def bpm(self, u, matchR, seen):

		# Try every node one by one
		for v in range(self.node_a):

			# If applicant u is interested
			# in job v and v is not seen
			if self.graph[u][v] and seen[v] == False:
				
				# Mark v as visited
				seen[v] = True
                
				if matchR[v] == -1 or self.bpm(matchR[v],
											matchR, seen):
					matchR[v] = u
					return True
		return False

	# Returns maximum number of matching
	def maxBPM(self):
		matchR = [-1] * self.node_b
		
		result = 0
		for i in range(self.node_a):
			
			seen = [False] * self.node_b
			
			if self.bpm(i, matchR, seen):
				result += 1
		return result

def solution(banana_list):
    graph = createGraph(banana_list)
    bmp = GFG(graph)
    max_match = bmp.maxBPM()
    if max_match % 2 == 0:
        ans =  len(banana_list) - max_match
    else:
        ans =  len(banana_list) - max_match + 1
    return ans


In [15]:
solution([4,3, 4, 3])

0

In [16]:
banana_list = [1, 7, 3, 21, 13, 19]
solution(banana_list)

0

# task 4.2

In [635]:
from __future__ import division
import math
from math import atan2
import itertools 

def mirrorCreator(room_dim, coords, dist):
    mirror = []
    for reflection in itertools.product(*[range(-(dist // -d) + 1) for d in room_dim]):
        for quad in [(1, 1), (-1, 1), (-1, -1), (1, -1)]:
            pos = tuple([
                (d * r + (d - c if r % 2 else c)) * q
                for d, c, r, q in zip(room_dim, coords, reflection, quad)
            ])
            mirror.append(pos)
    return list(set(mirror))

def dist_check(a,b):
    x,y = a[0],a[1]
    x2,y2 = b
    return ((x-x2)**2 + (y-y2)**2)**0.5

def reachCheck(me, mirror_other, mirror_me, dist):
    hit_me = {}
    for me_m in mirror_me:
        dist_m = dist_check(me, me_m)
        angle = getAngle(me, me_m)
        if (dist_m <= dist and angle not in hit_me) or (angle in hit_me and dist_m < abs(hit_me[angle])):
            hit_me[angle] = dist_m
    
    hit = {}
    for other_m in mirror_other:
        dist_m = dist_check(me, other_m)
        angle = getAngle(me, other_m)
        if ((dist_m <= dist) and (angle not in hit)):
            if angle not in hit_me or (angle in hit_me and dist_m < hit_me[angle]):
                hit[angle] = dist_m
        elif (angle in hit) and (dist_m < abs(hit[angle])):
            hit[angle] = dist_m
            
    return len(hit.keys())

def getAngle(me, other):
    ang = atan2(other[0]-me[0], other[1]-me[1])
    return ang

def solution(dimensions, your_position, trainer_position, distance):
    room_dim, me, other, dist = dimensions, your_position, trainer_position, distance
    mirror_me, mirror_other = mirrorCreator(room_dim, me, dist), mirrorCreator(room_dim, other, dist)
    hit= reachCheck(me, mirror_other, mirror_me, dist)
    return hit

In [636]:
dimensions, your_position, trainer_position, distance = [3,2], [1,1], [2,1], 4

In [638]:
solution(dimensions, your_position, trainer_position, distance)

7

## task 5

In [261]:
import decimal

def solution(n):
    n = int(n)
    decimal.getcontext().prec = 101
    sqrt2 = decimal.Decimal(2).sqrt()
    return str(calculation(n, sqrt2))
    
def calculation(n, sqrt2):
    if n == 0:
        return 0
    n_prime = int(n*(sqrt2-1))
    answer = n*n_prime + n*(n+1)/2 - n_prime*(n_prime+1)/2 - calculation(n_prime, sqrt2)
    return int(answer)

In [251]:
x = str(10**100)

In [260]:
solution(x)

'70710678118654752143350645116549947070840466124095050755453529494069282968337833969770392878674272517953662957768365280642962041509630934104758541368362044363113819683359518041450836040875813896192000'

In [253]:
solution1(x)

'70710678118654752143350645116549947070840466124095050755453529494069282968337833969770392878674272517953662957768365280642962041509630934104758541368362044363113819683359518041450836040875813896192000'

In [191]:

from decimal import *


def S(a, n):
    if n == 0:
        return 0
    np = int((a - 1) * n)
    return n * np + n * (n + 1) / 2 - np * (np + 1) / 2 - S(a, np)


def solution(s):
    getcontext().prec = 101
    return str(S(Decimal(2).sqrt(), int(s)))
